In [1]:
import pygame # this import here is just to prevent a strange bug (with some dynamic library used by pygame and other packages)

import sys
from os import path, getcwd
from IPython.display import clear_output

# Obtém o caminho do diretório do script atual
script_dir = getcwd()

# Adiciona o diretório pai ao caminho do sistema
sys.path.append(path.dirname(script_dir))

clear_output()

In [2]:
import time
from datetime import datetime
import collections

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.tensorboard import SummaryWriter
import tensorboard

import matplotlib.pyplot as plt

In [3]:
from modelos import dqn_models
from wrappers.atari_wrappers import *

ModuleNotFoundError: No module named 'util'

In [ ]:
torch.set_default_device(torch.device("cuda"))

In [ ]:
Experience = collections.namedtuple('Experience', field_names=['state', 'action', 'reward', 'done', 'new_state'])

class DQNExperienceBuffer:
    def __init__(self, capacity):
        self.buffer = collections.deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, s1, a, r, done, s2):
        experience = Experience(s1, a, r, done, s2)
        self.buffer.append(experience)

    def sample(self, batch_size):
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        states, actions, rewards, dones, next_states = zip(*[self.buffer[idx] for idx in indices])
        return np.array(states), np.array(actions), np.array(rewards, dtype=np.float32), \
               np.array(dones, dtype=np.uint8), np.array(next_states)